We are finally using the SFP algorithm in order to avoid the use of a universe list with all possible elements which creates a lot of false signals. It uses all the algorithms from the previous version, in addition, it uses SFP version on the top of the previous algorithms to give the really most frequent words, instead of a lot of fake counts.Unlike Apple's case, we will kinda maintain length 5 strings and will pad chars if not full.

In [ ]:
import collections
import hashlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
from collections import Counter

In [5]:
class Differential_Privacy_CMS_with_SFP():
    def __init__(self, width, depth, epsilon, max_coor=None):
        self.width = width
        self.depth = depth
        self.epsilon = epsilon
        self.seed = np.random.choice(np.arange(0,self.width),replace=False,size=self.depth) if width!=None else None
        self.max = max_coor
    def CLient_Side(self,d):
        '''parameters are a data element, epsilon value and a hash family, but we are using mmh3 here so didn't add that as a parameter'''
        d = d[0]*self.max+d[1]
        j = np.random.randint(self.depth) 
        z_vect = np.zeros((1,self.width))
        v = z_vect-1 
        hex_num = hashlib.sha256((str(self.seed[j])+str(d)).encode('utf-8')).hexdigest()
        int_hex = int(hex_num[:5],16)
        index = int_hex % self.width
        v[0,index]=1
        val = np.exp(self.epsilon/2)
        probability_of_1 = val/(val+1) 
        probability_of_neg_1 = 1/(val+1)
        b = np.random.choice([1,-1],self.width,p=[probability_of_1,probability_of_neg_1]) 
        final_vector = v*b
        return (final_vector,j)
        
    def Compute_Sketch_Matrix(self,D):
        '''So each element of D is a tuple where the first element is v_i and second element is j_i(which is created by the above funstion after getting 
        passed by Client_side each time), we have privacy parameter epsilon and dimensions. v_i is the a vector an it has the sma eshape as the 
        vectors returned by the Client_side algorithm'''
        val = np.exp(self.epsilon/2)
        n = len(D)
        c_epsilon = (val+1)/(val-1) 
        vec_one = np.ones((1,self.width))
        manipulated_data_matrix = np.zeros((n,self.width)) #Creatimg a matrix for x_is
        for elt in enumerate(D):
            new_vect = elt[1][0].reshape((1,self.width))
            man_vect = (c_epsilon/2)*new_vect #scalar and vector multiplication 
            half_vec_one = 0.5*vec_one 
            sum_vect = man_vect + half_vec_one
            manipulated_data_matrix[[elt[0]],:]= self.depth*sum_vect  
        M = np.zeros((self.depth,self.width))
        for elt in enumerate(D):
            for l in range(self.width): 
                M[elt[1][1],l]=M[elt[1][1],l]+manipulated_data_matrix[elt[0],l]
        return M 
        
    def Server_Side(self,Sketch_Matrix,d,length):
        '''It returns the estimated frequency of a data element given to it. So it has two parameters, data element and the length of the 
        data stream we are considering'''
        d=d[0]*self.max+d[1]
        n = length 
        frac1 = self.width/(self.width-1)
        frac2 = n/self.width
        row_sum = 0
        for i in range(self.depth):
            hex_num = hashlib.sha256((str(self.seed[i])+str(d)).encode('utf-8')).hexdigest()
            int_hex = int(hex_num[:5],16)
            index = int_hex % self.width
            row_sum = row_sum + Sketch_Matrix[i,index]
        avg_row_sum = row_sum/self.depth 
        subtraction = avg_row_sum - frac2 
        assumed_freq = frac1*subtraction 
        return assumed_freq

    def Count_Mean_Sketch(self,D_s,D,hash_eps_indicator): 
        '''D_s is the stream of data and this is a subset of universe of data'''
        Modified_datalist = []
        Sketch_Matrix = []
        freq_vect = {}
        length = len(D_s) 
        for elt in enumerate(D_s):
            Modified_datalist.append(self.CLient_Side(elt[1])) 
        Sketch_Matrix = self.Compute_Sketch_Matrix(Modified_datalist)
        for d in D:
            freq_vect[d] = self.Server_Side(Sketch_Matrix,d,length) 
        return freq_vect

    def frequency_counter_on_average(self,path_list):
        '''This function runs the entire process multiple and take the running average of frequencies for reducing error'''
        u_list = [(x,y) for x in range(20) for y in range(80)]
        freq_counter_cum = {d:0 for d in u_list}
        for i in range(500):
            freq_counter_new = self.Count_Mean_Sketch(path_list, u_list)
            for d in freq_counter_new:
                freq_counter_cum[d]=(i*freq_counter_cum[d]+freq_counter_new[d])/(i+1)
        return freq_counter_cum

def Client_SFP(d):
    '''d is a tuple in our example, so for converting it to a string, we are doing this d[0]*self.max+d[1] and then make it a string.'''
    l = np.random.randint(2)
    d =  str(d[0]*self.max+d[1])
    if len(d)<4:
        pad = 4-len(d)
        for i in range(pad):
            d+='0'
    output256 = hashlib.sha256(d.encode('utf-8')).hexdigest()#h(s)
    r = output256 + d[2*l:2*l+2]
    return (self.CLient_Side(r,'0'), self.Client_side(d,'main'),l)
def splitter(string):
    '''getting a string, it splits it into two parts, second part contains the last two characters.'''

 
def Server_SFP(server_elt_set,CMS_class_main,CMS_class_not,T):
    '''For all data elements client SFP creates a tuple of length 3 and call server SFP on all those elements.So D ois the set of all those tuples.'''
    beta_data = []
    for elt in server_elt_set:
        beta_data.append(elt[1])
    M = CMS_class_main.frequency_counter_on_average(beta_data)
    alpha_list1=[]
    alpha_list2=[]
    for elt in server_elt_set:
        if elt[2]==0:
            alpha_list1.append(elt[0])
        else:
            alpha_list2.append(elt[0])
    M1 = CMS_class_not.frequency_counter_on_average(alpha_list1)#frequency histogram
    M2 = CMS_class_not.frequency_counter_on_average(alpha_list2)#frequency histogram
    Q1=set(dict(sorted(M1.items(), key = itemgetter(1), reverse =True)[:T]).keys())
    Q2=set(dict(sorted(M2.items(), key = itemgetter(1), reverse =True)[:T]).keys())
    Q1_dict={}
    Q2_dict={}
    for elt in Q1:
        word,val = splitter(elt)
        Q1_dict[word]=val
    for elt in Q2:
        word,val=splitter(elt)
        Q2_dict[word]=val
    #will find the intersection of Q1 and Q2
    new_string_set = set()
    for word in Q1_dict:
        if word in Q2_dict:
            add_string = Q1_dict[word]+Q2_dict[word]
            new_string_set.add(add_string)
    final_freq_dict ={}
    for elt in new_string_set:
        final_freq_dict[elt]=M[elt]
    return final_freq_dict
    
def Sequence_Fragment_Puzzle(D_s,D,T):
    '''Main driver code of sequence fragment puzzle.'''
    CMS_class_main = Differential_Privacy_CMS_with_SFP(100,100,5,90)
    CMS_class_not = Differential_Privacy_CMS_with_SFP(100,100,5,90)
    T = 100 #threshold value, the maximum number 
    server_elt_set = set()
    for d in D_s:
        server_elt_set.add(Client_SFP(d))
    return Server_SFP(server_elt_set,CMS_class_main,CMS_class_not,T)
